准备测试数据

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor


In [ ]:
entro_data=pd.read_csv('smooth_entro.csv')
lonal_data=pd.read_csv('apply_lonal.csv')


In [ ]:
entro_data.head()

In [ ]:
lonal_data.head()

In [ ]:
df=pd.concat([entro_data,lonal_data],axis=1,join='inner')
df=df.T.drop_duplicates().T
df.describe()

In [ ]:
df.to_csv('lonal_entro.csv')

完毕，测试数据`lonal_entro.csv`

In [ ]:
df=pd.read_csv('lonal_entro.csv')
df.describe()

In [ ]:
df.columns


In [ ]:
准备尝试三种方案
1. 原数据对trip回归
2. 逻辑回归/决策树回归+回归，0-1矩阵乘以二次预测结果，从而同时预测od的发生和trip数量
3. 两次回归，矩阵相乘。
预测完成后，通过trip推算投送量

先尝试对od对进行0-1预测

In [ ]:
data=pd.read_csv('lonal_entro.csv')
model_1=RandomForestRegressor(random_state=1)
model_2a=RandomForestRegressor(random_state=1)
model_2b=RandomForestRegressor(random_state=1)



In [ ]:
datax=pd.read_csv('0_removed.csv')
trip=datax['trip_m6_wd_pm']
bins=[-0.1,0.5,5000]
cut_res=pd.cut(trip,bins=bins,labels=[0,1])
data['has_trip']=cut_res
data.to_csv('lonal_entro.csv')

In [ ]:
data.columns

In [ ]:
ycol='has_trip'
xcol=[ 'ori', 'dst','dist_km','fsize_m6_wd_pm','community_m6','mrt_km_o',
'far_hdb_o', 'far_priv_o', 'far_comm_o', 'mrt_km_d',
'far_hdb_d', 'far_priv_d', 'far_comm_d', 'smooth_entro_o',
       'smooth_entro_d','ori_lat', 'ori_lon', 'dst_lat', 'dst_lon',]

In [ ]:
y=data[ycol]
X=data[xcol]
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)
model_2a.fit(X=X_train,y=y_train)


In [ ]:
od01_pred=model_2a.predict(X_test)
r2=r2_score(y_pred=od01_pred,y_true=y_test)
r2


In [ ]:
loss=mean_squared_error(y_pred=od01_pred,y_true=y_test)
loss

In [ ]:
可以发现，随机森林能够非常好地对发生出行的od对进行预测
再尝试直接对需求进行预测

In [ ]:
ycol1='trip_m6_wd_pm'
xcol1=[ 'ori', 'dst','dist_km','fsize_m6_wd_pm','community_m6','mrt_km_o',
'far_hdb_o', 'far_priv_o', 'far_comm_o', 'mrt_km_d',
'far_hdb_d', 'far_priv_d', 'far_comm_d', 'smooth_entro_o',
       'smooth_entro_d']
y1=data[ycol1]
X1=data[xcol1]
X_train1,X_test1,y_train1,y_test1 = train_test_split(X1,y1,test_size=0.3)




In [ ]:
model_1.fit(X=X_train1,y=y_train1)

In [ ]:
trip_pred=model_1.predict(X_test1)
r2=r2_score(y_pred=trip_pred,y_true=y_test1)
r2

In [ ]:
loss2=mean_squared_error(y_pred=trip_pred,y_true=y_test1)
loss2

r2在0.5左右，效果一般。下面使用双层来预测。
model_2a:0-1
model_1:trip

In [ ]:
model_2a.predict

In [ ]:
double_lay_pred=od01_pred*trip_pred

In [ ]:
应该尝试为两层预测分别选择不同的合适的特征，而不是一股脑全丢进去